## Feature Development Extraction: Neurophysiology [dyskinesia project]




<b> Content </b>


<b> Aperiodic estimates </b>
Relevant literature:
- Periodic and a-periodic components relevance and interaction, different reasons (per + a-per) for signal changes observed within a specific bandwidth. Aperiodic component (complicated) vs exponent (1/f) (Donoghue, ..., Shestyuk & Voytek, Nature Neurosc 2020 : https://www.nature.com/articles/s41593-020-00744-x)
- cycle-by-cycle features: bycycle toolbox (Cole & Voytek, J of Neurophys 2019, https://journals.physiology.org/doi/full/10.1152/jn.00273.2019)
- aperiodic component, PD severity, and cortico-subcortico-activity, Bush & Zou, Richardson, bioRxiv 2023 https://www.biorxiv.org/content/10.1101/2023.02.08.527719v1?rss=1

<b> Periodic component analysis: </b> 
- Try Wavelet Dceomposition vs Welch (tapered) Spectral Decomposition





### 0. Loading packages and functions, defining paths



In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import pandas as pd
import numpy as np
import sklearn as sk

import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version



In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
# print('mne_bids', mne_bids.__version__)
# print('mne', mne.__version__)
# print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)
print('matplotlib', plt_version)
## FEB 2022:
# Python sys 3.9.7 (default, Sep 16 2021, 08:50:36) 
# [Clang 10.0.0 ]
# pandas 1.3.4
# numpy 1.20.3
# mne_bids 0.9
# mne 0.24.1
# sci-py 1.7.1
# sci-kit learn 1.0.1

In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')

In [ ]:
os.chdir(codepath)
# own utility functions
import utils.utils_fileManagement as utilsFiles
import utils.utils_windowing as utilsWindows
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)

# own data exploration functions
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_analysis.ft_processing_helpers as ftProc

import lfpecog_plotting.plotHelpers as plotHelp

### Plot ssd'd Spectral Descriptives

#### SSD time freq plotting


In [ ]:
import lfpecog_plotting.plot_timeFreqs_ssd_psds as plot_ssd_TFs

In [ ]:
WIN_LEN=10
WIN_OVERLAP=0.5
DATA_VERSION='v4.0'
FT_VERSION='v6'
SSD_BROAD=True
INCL_STN_ONLY_PTS=True
IGNORE_PTS=['011', '104', '106']

In [ ]:
importlib.reload(ftProc)
importlib.reload(importClin)
importlib.reload(plot_ssd_TFs)

# CONVERT ONLY STNS!!!!!
FIG_DATE = '0000'
SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                          FT_VERSION=FT_VERSION,
                          IGNORE_PTS=IGNORE_PTS)
print(SUBS)

for sub in SUBS:
    if sub not in ['023',]:
        print(f'skip {sub}')
        continue
    # # try:
    print(f'PLOT SUB-{sub}')
    plot_ssd_TFs.plot_indiv_ssd_timefreq_allSources(
        sub, fig_name_base=f'{FIG_DATE}_ssdTimeFreq',
        FT_VERSION=FT_VERSION,
        DATA_VERSION=DATA_VERSION,
        ZSCORE=False, LOG_POWER=True,
        SAVE_PLOT=True)
    # except:
    #     print(f'sub {sub} error')

#### SSD PSD plotting

In [ ]:

import lfpecog_analysis.get_SSD_timefreqs as ssd_TimeFreq


In [ ]:
# WIN_LEN=10
# WIN_OVERLAP=0.5
# DATA_VERSION='v4.2'
# FT_VERSION='v7'
# SSD_BROAD=True
# INCL_STN_ONLY_PTS=True
# IGNORE_PTS=['011', '104', '106']

In [ ]:
SUBS = utilsFiles.get_avail_ssd_subs(DATA_VERSION=DATA_VERSION,
                          FT_VERSION=FT_VERSION,
                          IGNORE_PTS=IGNORE_PTS)
print(SUBS)
print(len(SUBS))

In [ ]:
importlib.reload(ssd_TimeFreq)


TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(
    SUBS=SUBS, FT_VERSION=FT_VERSION,
    DATA_VERSION=DATA_VERSION,
)


In [ ]:
import lfpecog_analysis.get_acc_task_derivs as accDerivs
import lfpecog_plotting.plot_descriptive_SSD_PSDs as plot_PSDs
import lfpecog_plotting.plot_COH_spectra as plot_COHs

In [ ]:
importlib.reload(ftProc)

importlib.reload(ssd_TimeFreq)
importlib.reload(accDerivs)

### Plot Coherence Spectra Densities

In [ ]:
importlib.reload(plot_COHs)

fig, axes = plt.subplots(2, 2, figsize=(12, 12))

for i_row, COH_type in enumerate(['sq_coh', 'imag_coh']):
    
    for i_col, MOV in enumerate(['INCL_MOVE', 'EXCL_MOVE']):

        axes[i_row, i_col] = plot_COHs.plot_COH_spectra(
            COH_type=COH_type, COH_source='STN_ECOG',
            SELECT_ON_ACC_RMS=MOV,
            RETURN_AX=True,
            given_ax=axes[i_row, i_col]
        )

plt.show()

##### Plot Group Results: Spectral changes over Time after L-Dopa, non-LID versus LID

In [ ]:
### TODO: put in function in script

# make groups with and without occurence of LID
subs_LID = []
subs_noLID = []

for sub in SUBS:
    print(f'check sub-{sub}')
    if sub in IGNORE_PTS:
        
        print(f'skip sub-{sub}')
        continue
    try:
        max_score = max(ftProc.get_cdrs_specific(sub=sub,
                                                rater='Jeroen',
                                                side='both')[1])
        if max_score > 0: subs_LID.append(sub)
        else: subs_noLID.append(sub)
    except ValueError:
        print(f'sub-{sub} not included, no CDRS scores')

In [ ]:
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

SAVE=True
SHOW=False

SAVE_DATE = '1023'
STN_or_ECOG='STN'
ZSCORE_FREQS=False
SMOOTH_FREQS=6
LOG_POWER=False
BASELINE_CORRECT=True

n_subs_incl = len(subs_LID) + len(subs_noLID)

fig_name = f'{SAVE_DATE}_{STN_or_ECOG}_PSDs_noLID_LID_vs_DopaTime_n{n_subs_incl}'
if BASELINE_CORRECT: fig_name += '_blCorrPrc'
if ZSCORE_FREQS: fig_name += '_Z'
if LOG_POWER: fig_name += '_log'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'


# plot_PSD_vs_DopaTime(TFs['008']['lfp_left'])
fig, axes = plt.subplots(1, 2, figsize=(16, 8))
fsize=20

# if None returned, dont use axes[0] to collect returned output
if STN_or_ECOG == 'STN': ax_title = 'Bilateral STNs w/o Dyskinesia'
elif STN_or_ECOG == 'ECOG': ax_title = 'ECoG w/o Dyskinesia'
axes[0] = plot_PSDs.plot_PSD_vs_DopaTime(
    TFs, sel_subs=subs_noLID,
    STN_or_ECOG=STN_or_ECOG,
    BASELINE_CORRECT=BASELINE_CORRECT,
    ZSCORE_FREQS=ZSCORE_FREQS,
    LOG_POWER=LOG_POWER,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    plt_ax_to_return=axes[0], fsize=fsize, BREAK_X_AX=True,
    ax_title=ax_title
)

if STN_or_ECOG == 'STN': ax_title = 'Bilateral STNs with Dyskinesia'
elif STN_or_ECOG == 'ECOG': ax_title = 'ECoG with Dyskinesia'
axes[1] = plot_PSDs.plot_PSD_vs_DopaTime(
    TFs, sel_subs=subs_LID,
    STN_or_ECOG=STN_or_ECOG,
    LOG_POWER=LOG_POWER,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    BASELINE_CORRECT=BASELINE_CORRECT,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[1], fsize=fsize, BREAK_X_AX=True,
    ax_title=ax_title
)


# equalize axes
ymin = min([min(ax.get_ylim()) for ax in axes])
ymax = max([max(ax.get_ylim()) for ax in axes])
for ax in axes: ax.set_ylim(ymin, ymax)

for ax in axes: ax.tick_params(axis='both', size=fsize,
                               labelsize=fsize)
plt.tight_layout()

if SAVE:
    path = os.path.join(get_project_path('figures'),
                        'ft_exploration',
                        f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                        'descr_PSDs')
    if not os.path.exists(path): os.makedirs(path)
    plt.savefig(os.path.join(path, fig_name),
                facecolor='w', dpi=300,)
if SHOW: plt.show()
else: plt.close()



##### Plot LID-Group Results: Laterality of LID in STN and ECoG

Plot SEVERITY AND LATERALITY OF STN-changes during DYSKINESIA

- LAT_UNI: plot laterality of unilateral LID
- LAT_BILAT: plot laterality of only bilateral LID
- SCALE: combine all uni- and bilateral LID
    - PLOT_ONLY_MATCH == True: plot only STN contralateral to LID
    - PLOT_ONLY_MATCH == False: plot contra-, ipsi-, and bi-lateral STN-LID


In [ ]:
import lfpecog_analysis.psd_lid_stats as psd_stats
import lfpecog_analysis.get_acc_task_derivs as accDerivs

In [ ]:
importlib.reload(accDerivs)
importlib.reload(importClin)
importlib.reload(plotHelp)
importlib.reload(psd_stats)
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

# 'LAT_UNI' , 'LAT_BILAT', 'SCALE', 'LAT_ALL_SCALE'
LAT_or_SCALE = 'SCALE'
STAT_DATE = '1023'
FIG_DATE = '1023'

INCL_PRELID = False
PRELID_MINs=5
SMOOTH_FREQS=8
LOG_POWER=False
BASELINE_CORRECT=True
ZSCORE_FREQS=False
SHOW_GAMMA = False
SHOW_SIGN = False
PLOT_ONLY_MATCH = True
SELECT_ACC_RMS='EXCL_MOVE'

fig_name = f'{FIG_DATE}_STN_PSDs_{LAT_or_SCALE}_LID_n{len(SUBS)}'

if INCL_PRELID: fig_name += f'_inclPre{PRELID_MINs}'
if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'
# if SINGLE_LINE: fig_name += '_lines'
if SHOW_GAMMA: fig_name += '_GAMMA'
if SHOW_SIGN: fig_name += '_SIGN'
if PLOT_ONLY_MATCH: fig_name += '_onlyMatch'
if SELECT_ACC_RMS: fig_name += f'_{SELECT_ACC_RMS}'


fsize=14

stn_stats = plot_PSDs.plot_STN_PSD_vs_LID(
    all_timefreqs=TFs,
    sel_subs=subs_LID,
    CDRS_RATER='Jeroen',
    LAT_or_SCALE=LAT_or_SCALE,
    SELECT_ON_ACC_RMS=SELECT_ACC_RMS,
    incl_PRE_LID=INCL_PRELID,
    PRELID_MIN=PRELID_MINs,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=False,
    fsize=fsize,
    BREAK_X_AX=True,
    fig_name=fig_name,
    RETURN_FREQ_CORR=False,
    SHOW_ONLY_GAMMA=SHOW_GAMMA,
    SHOW_SIGN=SHOW_SIGN,
    PROCESS_STATS=False,
    p_SAVED_DATE=STAT_DATE,
    PLOT_ONLY_MATCH=PLOT_ONLY_MATCH,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)


Plot SEVERITY AND LATERALITY OF ECoG-changes during DYSKINESIA

In [ ]:
importlib.reload(accDerivs)
# importlib.reload(psdLID_stats)
importlib.reload(plot_PSDs)

# PM, needs: TFs = ssd_TimeFreq.get_all_ssd_timeFreqs(SUBS=SUBS)

# 'LAT_UNI' , 'LAT_BILAT', 'SCALE', 'LAT_ALL_SCALE'
FIG_DATE = '1023'
STAT_DATE = '1023'
LAT_or_SCALE = 'SCALE'
SAVE_PLOT = True
SHOW_PLOT = False

SMOOTH_FREQS = 8
LOG_POWER = False
BASELINE_CORRECT = True
ZSCORE_FREQS = False
SINGLE_LINE = False
SHOW_GAMMA = False
PLOT_ONLY_MATCH = True
SHOW_SIGN = False
SELECT_ACC_RMS='INCL_MOVE'

fig_name = f'{FIG_DATE}_ECoG_PSDs_LID_{LAT_or_SCALE}_n{len(SUBS)}'

if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'
if SINGLE_LINE: fig_name += '_lines'
if SHOW_GAMMA: fig_name += '_GAMMA'
if SHOW_SIGN: fig_name += '_SIGN'
if PLOT_ONLY_MATCH: fig_name += '_onlyMatch'
if SELECT_ACC_RMS: fig_name += f'_{SELECT_ACC_RMS}'

fsize=14

plot_PSDs.plot_ECOG_PSD_vs_LID(
    TFs, sel_subs=subs_LID,
    LAT_or_SCALE=LAT_or_SCALE,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SELECT_ON_ACC_RMS=SELECT_ACC_RMS,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=False,
    fsize=fsize,
    BREAK_X_AX=True,
    fig_name=fig_name,
    single_sub_lines=SINGLE_LINE,
    PLOT_ONLY_MATCH=PLOT_ONLY_MATCH,
    SHOW_ONLY_GAMMA=SHOW_GAMMA,
    SHOW_SIGN=SHOW_SIGN,
    PROCESS_STATS=False,
    p_SAVED_DATE=STAT_DATE,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)

Plot LATERALITY during UNILATERAL dyskinesia

In [ ]:
importlib.reload(plot_PSDs)


SAVE_PLOT = True
SHOW_PLOT = False

SMOOTH_FREQS=8
LOG_POWER=False
BASELINE_CORRECT=True
ZSCORE_FREQS=False
CDRS_RATER = 'Jeroen'

fig_name = f'1002_LATERALITY_STNandECOG_PSDs_unilatLID_n{len(SUBS)}'
if BASELINE_CORRECT: fig_name += '_blCorr'
if LOG_POWER: fig_name += '_log'
if ZSCORE_FREQS: fig_name += '_Z'
if SMOOTH_FREQS > 0: fig_name += f'_smooth{SMOOTH_FREQS}'



fig, axes = plt.subplots(1, 2, figsize=(12, 6))

fsize=14

plot_PSDs.plot_STN_PSD_vs_LID(
    TFs, sel_subs=subs_LID,
    LAT_or_SCALE='LAT_UNI',  # LAT_UNI
    CDRS_RATER=CDRS_RATER,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[0],
    fsize=fsize,
    BREAK_X_AX=True,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)
plot_PSDs.plot_ECOG_PSD_vs_LID(
    TFs, sel_subs=subs_LID,
    LAT_or_SCALE='LAT_UNI',
    CDRS_RATER=CDRS_RATER,
    LOG_POWER=LOG_POWER,
    BASELINE_CORRECT=BASELINE_CORRECT,
    SMOOTH_PLOT_FREQS=SMOOTH_FREQS,
    ZSCORE_FREQS=ZSCORE_FREQS,
    plt_ax_to_return=axes[1],
    fsize=fsize,
    BREAK_X_AX=True,
    DATA_VERSION=DATA_VERSION,
    FT_VERSION=FT_VERSION,
)

# equalize axes
ymin = min([min(ax.get_ylim()) for ax in axes])
ymax = max([max(ax.get_ylim()) for ax in axes])
for ax in axes: ax.set_ylim(ymin, ymax)

for ax in axes: ax.tick_params(axis='both', size=fsize,
                               labelsize=fsize)
plt.tight_layout()

if SAVE_PLOT:
    plt.savefig(os.path.join(get_project_path('figures'),
                             'ft_exploration',
                             f'data_{DATA_VERSION}_ft_{FT_VERSION}',
                             'descr_PSDs', fig_name),
                facecolor='w', dpi=300,)
if SHOW_PLOT: plt.show()
else: plt.close()